A simple colab to fine-tune openai diffusion models.


Feel free to ask questions in this post's comments: https://www.patreon.com/posts/66246423

by [Alex Spirin](https://twitter.com/devdef)

![visitors](https://visitor-badge.glitch.me/badge?page_id=sxela_finetune_openai_colab)

#Train (tune) BEDROOM model :D
Needs 16gb GPU RAM

Works in colab pro and on kaggle 

## Setup (run once per session)

This mounts your google drive for easier storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This downloads the training code and installs it, then downloads a pre-trained model that we will be tuning on our dataset

In [ ]:
%cd /content
!git clone https://github.com/Sxela/guided-diffusion-sxela
%cd /content/guided-diffusion-sxela
!pip install -e .

!wget https://openaipublic.blob.core.windows.net/diffusion/march-2021/lsun_uncond_100M_1200K_bs128.pt -P /content/

28.pt                25%[====>               ] 110.18M  15.9MB/s    eta 28s    ^C


## Tune

For gigachads. 
We're going to do what's called a pro-gamer move (or not): tune a small model, trained on bedrooms, on our own dataset. Just because we can and it's much faster than training from scratch. 

Don't forget to change the paths:
You need to change DATASET_PATH to point to your dataset images folder, and CHECKPOINT_PATH - to point to a folder you'd like to save progress to. 

For, example here /content/drive/MyDrive/deep_learning/guided-diffusion-sxela/ - this path points to a location, where all the training checkpoints will be saved

and /content/YourDatasetHere/ - this path points to your dataset, i.e. a folder with images (no captions needed)




We will be using this model together with CLIP inside DiscoDiffusion, so we can train less, stop early and let CLIP do the heavy lifting.

This will run almost forever, but you should start checking your results at around ~50k iterations. Good results begin to appear at 100-200k iterations, depending on your dataset.

Validating means opening your CHECKPOINT_PATH folder, taking the ema_0.9999_(some number of steps).pt file with the highest number (the latest one), going to this version of DiscoDiffusion here 
https://github.com/Sxela/DiscoDiffusion-Warp/blob/main/Disco_Diffusion_v5_2_Warp_custom_model.ipynb and setting this: diffusion-model - custom, custom_path - path to that ema file from the previous step (if you saved it on google drive - then just point it there), and set width_height to 256x256, then run DD as usual


In [ ]:
MODEL_FLAGS="--image_size 256 --num_channels 128 --num_res_blocks 2 --num_heads 1 --learn_sigma True --use_scale_shift_norm False --attention_resolutions 16"
DIFFUSION_FLAGS="--diffusion_steps 1000 --noise_schedule linear --rescale_learned_sigmas False --rescale_timesteps False --use_scale_shift_norm False"
TRAIN_FLAGS="--lr 2e-5 --batch_size 4 --save_interval 2000 --log_interval 50 --resume_checkpoint /content/lsun_uncond_100M_1200K_bs128.pt"
DATASET_PATH="/content/YourDatasetHere/" #change to point to your dataset path 
CHECKPOINT_PATH="/content/drive/MyDrive/deep_learning/guided-diffusion-sxela/"
%cd /content/guided-diffusion-sxela
!OPENAI_LOGDIR=$CHECKPOINT_PATH python scripts/image_train.py --data_dir $DATASET_PATH $MODEL_FLAGS $DIFFUSION_FLAGS $TRAIN_FLAGS

## Sampling
The best way to sample your model in real-life conditions is to plug it into DiscoDiffusion. 


Grab your latest ema checkpoint, open this colab here - https://github.com/Sxela/DiscoDiffusion-Warp/blob/main/Disco_Diffusion_v5_2_Warp_custom_model.ipynb

and change model settings > custom model path to your ema checkpoint's location, as described in the previous cell.

You can still sample using vanilla openai code, just plug your checkpoint in the cell below

Don't forget to change all the paths

In [ ]:
checkpoint_path = 'input some checkpoint path here' #use ema checkpoint
!OPENAI_LOGDIR=/content/drive/MyDrive/deep_learning/guided-diffusion-sxela/samples/  python scripts/image_sample.py --num_samples 1 --model_path $checkpoint_path $MODEL_FLAGS $DIFFUSION_FLAGS --timestep_respacing ddim100

In [ ]:
import numpy as np
import PIL

sample_path = 'some sample path'
im = np.load(sample_path)
PIL.Image.fromarray(im.f.arr_0[0])

#Train (tune) 256x256 vanilla DD model
Only if you have a beefy GPU with more than 16gb RAM

For lvl 50 AI bosses, 
Will not fit into colab pro, only in colab pro+ with A100 gpu


## Setup

This mounts your google drive for easier storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

This downloads the training code and installs it, then downloads a pre-trained model that we will be tuning on our dataset.

I'm no using my edition of guided-diffusion in case you're going to use multiple GPUs

In [ ]:
%cd /content
!git clone https://github.com/openai/guided-diffusion
%cd /content/guided-diffusion  
!pip install -e .
!pip install mpi4py 
#is using on kaggle, replace !pip install mpi4py  with !conda install -y mpi4py
#download model checkpoint
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt -P /content/



## Tune

Don't forget to change the paths:
You need to change DATASET_PATH to point to your dataset images folder, and CHECKPOINT_PATH - to point to a folder you'd like to save progress to. 

For, example here /content/drive/MyDrive/deep_learning/guided-diffusion-sxela/ - this path points to a location, where all the training checkpoints will be saved

and /content/YourDatasetHere/ - this path points to your dataset, i.e. a folder with images (no captions needed)




We will be using this model together with CLIP inside DiscoDiffusion, so we can train less, stop early and let CLIP do the heavy lifting.

This will run almost forever, but you should start checking your results at around ~50k iterations. Good results begin to appear at 100-200k iterations, depending on your dataset.

Validating means opening your CHECKPOINT_PATH folder, taking the ema_0.9999_(some number of steps).pt file with the highest number (the latest one), going to this version of DiscoDiffusion here 
https://github.com/Sxela/DiscoDiffusion-Warp/blob/main/Disco_Diffusion_v5_2_Warp_custom_model.ipynb and setting this: diffusion-model - custom, custom_path - path to that ema file from the previous step (if you saved it on google drive - then just point it there), 

you'll need to set custom model settings to this: 

    model_config.update({
        'attention_resolutions': '32, 16, 8',
        'class_cond': False,
        'diffusion_steps': diffusion_steps,
        'rescale_timesteps': True,
        'timestep_respacing': timestep_respacing,
        'image_size': 256,
        'learn_sigma': True,
        'noise_schedule': 'linear',
        'num_channels': 256,
        'num_head_channels': 64,
        'num_res_blocks': 2,
        'resblock_updown': True,
        'use_checkpoint': use_checkpoint,
        'use_fp16': True,
        'use_scale_shift_norm': True,
    })

In [ ]:
MODEL_FLAGS="--attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 256 --num_head_channels 64  --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True"
TRAIN_FLAGS="--lr 2e-5 --batch_size 4 --save_interval 1000 --log_interval 50 --resume_checkpoint /content/256x256_diffusion_uncond.pt"  
DATASET_PATH="/content/YourDatasetHere/" #change to point to your dataset path 
CHECKPOINT_PATH="/content/drive/MyDrive/deep_learning/guided-diffusion/"
%cd /content/guided-diffusion
!OPENAI_LOGDIR=$CHECKPOINT_PATH python scripts/image_train.py --data_dir $DATASET_PATH $MODEL_FLAGS $TRAIN_FLAGS

Sample from model

## Sampling
The best way to sample your model in real-life conditions is to plug it into DiscoDiffusion. 


Grab your latest ema checkpoint, open this colab here - https://github.com/Sxela/DiscoDiffusion-Warp/blob/main/Disco_Diffusion_v5_2_Warp_custom_model.ipynb

and change settings like described in the previous cell

You can still sample using vanilla openai code, just plug your checkpoint in the cell below

Don't forget to change all the paths

In [ ]:
checkpoint_path = 'input some checkpoint path here'
!OPENAI_LOGDIR=/content/samples/  python scripts/image_sample.py --num_samples 1 --model_path $checkpoint_path $MODEL_FLAGS --timestep_respacing ddim100

Show results

In [ ]:
import numpy as np
import PIL

sample_path = 'some sample path'
im = np.load(sample_path)
PIL.Image.fromarray(im.f.arr_0[0])

# Train from scratch (smaller model)
For lvl 1 AI crooks like me, should fit into colab pro

Train a smaller model that will fit definitely into colab pro.

Don't forget to change the paths:
You need to change DATASET_PATH to point to your dataset images folder, and CHECKPOINT_PATH - to point to a folder you'd like to save progress to. 

For, example here /content/drive/MyDrive/deep_learning/guided-diffusion-sxela/ - this path points to a location, where all the training checkpoints will be saved

and /content/YourDatasetHere/ - this path points to your dataset, i.e. a folder with images (no captions needed)




We will be using this model together with CLIP inside DiscoDiffusion, so we can train less, stop early and let CLIP do the heavy lifting.

This will run almost forever, but you should start checking your results at around ~50k iterations. Good results begin to appear at 100-200k iterations, depending on your dataset.

Validating means opening your CHECKPOINT_PATH folder, taking the ema_0.9999_(some number of steps).pt file with the highest number (the latest one), going to this version of DiscoDiffusion here 
https://github.com/Sxela/DiscoDiffusion-Warp/blob/main/Disco_Diffusion_v5_2_Warp_custom_model.ipynb and setting this: diffusion-model - custom, custom_path - path to that ema file from the previous step (if you saved it on google drive - then just point it there), 

you'll need to set custom model settings to this: 

    model_config.update({
        'attention_resolutions': '32, 16, 8',
        'class_cond': False,
        'diffusion_steps': diffusion_steps,
        'rescale_timesteps': True,
        'timestep_respacing': timestep_respacing,
        'image_size': 256,
        'learn_sigma': True,
        'noise_schedule': 'linear',
        'num_channels': 128,
        'num_heads': 4,
        'num_res_blocks': 2,
        'resblock_updown': True,
        'use_checkpoint': use_checkpoint,
        'use_fp16': True,
        'use_scale_shift_norm': True,
    })

In [ ]:
MODEL_FLAGS="--attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 128 --num_heads 4  --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True"
TRAIN_FLAGS="--lr 2e-5 --batch_size 4 --save_interval 1000 --log_interval 50"
DATASET_PATH="/content/YourDatasetHere/" #change to point to your dataset path 
CHECKPOINT_PATH="/content/drive/MyDrive/deep_learning/guided-diffusion-sxela/"
%cd /content/guided-diffusion-sxela
!OPENAI_LOGDIR=$CHECKPOINT_PATH python scripts/image_train.py --data_dir $DATASET_PATH $MODEL_FLAGS $TRAIN_FLAGS

### Sampling
The best way to sample your model in real-life conditions is to plug it into DiscoDiffusion. 


Grab your latest ema checkpoint, open this colab here - https://github.com/Sxela/DiscoDiffusion-Warp/blob/main/Disco_Diffusion_v5_2_Warp_custom_model.ipynb

and change settings like described in the previous cell

In [ ]:
checkpoint_path = 'input some checkpoint path here'
!OPENAI_LOGDIR=/content/samples/  python scripts/image_sample.py --num_samples 1 --model_path $checkpoint_path $MODEL_FLAGS --timestep_respacing ddim100

Show results

In [ ]:
import numpy as np
import PIL

sample_path = 'some sample path'
im = np.load(sample_path)
PIL.Image.fromarray(im.f.arr_0[0])